# NOAA Web Scraper

A webscraper that automatically donwloads netCDF files from the [NOAA Sea Surface Tempterature Dataset](https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/). This dataset contains SST (Sea Surface Temperature) values for each day from approximately the 1980s till today resulting in over 14'000 files. 

The notebook was created in the context of the modules Think 4 / Play 4 in Spring semester 2025 at Data Design + Art by Max Frischknecht for a project by Elisee Kukulu and Eva Brinksma about the relation between hurricane intensity and sea surface temperature. 

## Basics

The notebook uses the Python packages `os` (for accessing the computers files and directories), `requests` (for opening websites), `BeatifulSoup` (for scraping data from websites) and `tqdm` to save the netCDF files. 

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# Base URL of the dataset
BASE_URL = "https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/" 

Start and ending can be defined to enable dowloading in batches, or restarting from a certain point. 

In [3]:
# Define start and end year
START_YEAR = 1990
END_YEAR = 1992
SAVE_DIR = "./export"  # Folder to save files

## Helper Functions  

The following functions looks up all download links from the provided website and returns them. It filters unnecessary links that are about site navigation and unrelated to the desired files. 

In [1]:
def get_links(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    # bet all the `href` content from all html `a` tags
    links = [a["href"] for a in soup.find_all("a", href=True)]

    # Remove unwanted links
    ignored_links = {'?C=N;O=D', '?C=M;O=A', '?C=S;O=A', '?C=D;O=A', '/data/sea-surface-temperature-optimum-interpolation/v2.1/access/'}
    filtered_links = [link for link in links if link not in ignored_links]
    
    # return the links
    return filtered_links

In [ ]:
# call the function
year_month_links = get_links(BASE_URL)
year_month_links

['198109/',
 '198110/',
 '198111/',
 '198112/',
 '198201/',
 '198202/',
 '198203/',
 '198204/',
 '198205/',
 '198206/',
 '198207/',
 '198208/',
 '198209/',
 '198210/',
 '198211/',
 '198212/',
 '198301/',
 '198302/',
 '198303/',
 '198304/',
 '198305/',
 '198306/',
 '198307/',
 '198308/',
 '198309/',
 '198310/',
 '198311/',
 '198312/',
 '198401/',
 '198402/',
 '198403/',
 '198404/',
 '198405/',
 '198406/',
 '198407/',
 '198408/',
 '198409/',
 '198410/',
 '198411/',
 '198412/',
 '198501/',
 '198502/',
 '198503/',
 '198504/',
 '198505/',
 '198506/',
 '198507/',
 '198508/',
 '198509/',
 '198510/',
 '198511/',
 '198512/',
 '198601/',
 '198602/',
 '198603/',
 '198604/',
 '198605/',
 '198606/',
 '198607/',
 '198608/',
 '198609/',
 '198610/',
 '198611/',
 '198612/',
 '198701/',
 '198702/',
 '198703/',
 '198704/',
 '198705/',
 '198706/',
 '198707/',
 '198708/',
 '198709/',
 '198710/',
 '198711/',
 '198712/',
 '198801/',
 '198802/',
 '198803/',
 '198804/',
 '198805/',
 '198806/',
 '198807/',
 '19

A second function that takes the prepared links and downloads each netCDF file. 

In [6]:
def download_file(url, save_path):
    """Downloads a file with a progress bar."""
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    total_size = int(response.headers.get("content-length", 0))
    with open(save_path, "wb") as file, tqdm(
        desc=os.path.basename(save_path),
        total=total_size,
        unit="B",
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
            bar.update(len(chunk))

## Data Collection

The main code that loops over all the links found and downloads the files from each link. The files are saved in the same folder structure as on the website: `YEARMONTH/FILES-BY-DAY.nc`

In [ ]:
for link in year_month_links:
     # Ensure it's a directory
    if link.endswith("/"): 

        # Extract year-month as folder name
        year_month = link.strip('/')

        # get the current year from the links for start/stop 
        year = int(link[:4])
        if START_YEAR <= year <= END_YEAR:

            # create the download link
            month_url = BASE_URL + link
            print(f"Processing: {month_url}")

            # Create subdirectory for year/month
            subfolder_path = os.path.join(SAVE_DIR, year_month)
            os.makedirs(subfolder_path, exist_ok=True)

             # Get all netCDF files of the current the month
            netcdf_files = get_links(month_url)

            # donwload the netcdf files of that month
            for file in netcdf_files:
                # Ensure it's a NetCDF file
                if file.endswith(".nc"):
                    # create the file url and save path
                    file_url = month_url + file
                    save_path = os.path.join(subfolder_path, file)
                    # check if the path/file already was downloaded
                    if not os.path.exists(save_path):
                        print(f"Downloading: {file_url}")
                        download_file(file_url, save_path)
                    else:
                        print(f"Already downloaded: {file}")          
print("Download complete!")

Processing: https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/199001/
Already downloaded: oisst-avhrr-v02r01.19900101.nc
Already downloaded: oisst-avhrr-v02r01.19900102.nc
Already downloaded: oisst-avhrr-v02r01.19900103.nc
Already downloaded: oisst-avhrr-v02r01.19900104.nc
Already downloaded: oisst-avhrr-v02r01.19900105.nc
Already downloaded: oisst-avhrr-v02r01.19900106.nc
Already downloaded: oisst-avhrr-v02r01.19900107.nc
Already downloaded: oisst-avhrr-v02r01.19900108.nc
Already downloaded: oisst-avhrr-v02r01.19900109.nc
Already downloaded: oisst-avhrr-v02r01.19900110.nc
Already downloaded: oisst-avhrr-v02r01.19900111.nc
Already downloaded: oisst-avhrr-v02r01.19900112.nc
Already downloaded: oisst-avhrr-v02r01.19900113.nc
Already downloaded: oisst-avhrr-v02r01.19900114.nc
Already downloaded: oisst-avhrr-v02r01.19900115.nc
Already downloaded: oisst-avhrr-v02r01.19900116.nc
Already downloaded: oisst-avhrr-v02r01.19900117.nc
Already downloaded

oisst-avhrr-v02r01.19920901.nc: 100%|██████████| 1.64M/1.64M [00:00<00:00, 1.79MB/s]


Downloading: https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/199209/oisst-avhrr-v02r01.19920902.nc


oisst-avhrr-v02r01.19920902.nc: 100%|██████████| 1.64M/1.64M [00:00<00:00, 2.23MB/s]


Downloading: https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/199209/oisst-avhrr-v02r01.19920903.nc


...

Downloading: https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/199212/oisst-avhrr-v02r01.19921230.nc


oisst-avhrr-v02r01.19921230.nc: 100%|██████████| 1.62M/1.62M [00:00<00:00, 2.16MB/s]


Downloading: https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/199212/oisst-avhrr-v02r01.19921231.nc


oisst-avhrr-v02r01.19921231.nc: 100%|██████████| 1.62M/1.62M [00:00<00:00, 2.16MB/s]

Download complete!
